In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from StatusMsg import StatusMsg
import json
import os
import io
import datetime
import timedelta

In [12]:
def india(state,date):
    # URL = "https://www.mygov.in/corona-data/covid19-statewise-status/"
    # file_name, headers = urllib.request.urlretrieve(URL)
    soup = BeautifulSoup(open("../INPUT/"+date+"/TT_State.html", encoding="utf8"), "html.parser")

    # sum_URL = "https://www.mygov.in/covid-19"
    # sum_file_name, sum_headers = urllib.request.urlretrieve(sum_URL)
    sum_soup = BeautifulSoup(open("../INPUT/"+date+"/TT.html", encoding="utf8"), "html.parser")

    STATES = soup.find_all("div", {"class": "field field-name-field-select-state field-type-list-text field-label-above"})
    CONFIRMED = soup.find_all("div", {"class": "field field-name-field-total-confirmed-indians field-type-number-integer field-label-above"})
    CURED_DISCHARGED = soup.find_all("div", {"class": "field field-name-field-cured field-type-number-integer field-label-above"})
    DEATH = soup.find_all("div", {"class": "field field-name-field-deaths field-type-number-integer field-label-above"})

    states = []
    for val in STATES:
        states.append(str(val.getText()).split(":")[1].lstrip().replace("Telengana", "Telangana").replace('Andaman And Nicobar', 'Andaman and Nicobar Islands').replace('Andaman and Nicobar','Andaman and Nicobar Islands'))

    confirmed = []
    for val in CONFIRMED:
        confirmed.append(str(val.getText()).split(":")[1].lstrip())

    cured = []
    for val in CURED_DISCHARGED:
        cured.append(str(val.getText()).split(":")[1].lstrip())

    death = []
    for val in DEATH:
        death.append(str(val.getText()).split(":")[1].lstrip())

    states_data = pd.DataFrame(list(zip(states, confirmed, cured, death)))

    final_df_col = [
        'Date', 'State/UTCode', 'District', 'tested_last_updated_district', 'tested_source_district',
        'notesForDistrict', 'cumulativeConfirmedNumberForDistrict', 'cumulativeDeceasedNumberForDistrict',
        'cumulativeRecoveredNumberForDistrict', 'cumulativeTestedNumberForDistrict',
        'cumulativeVaccinatedNumberForDistrict', 'last_updated', 'tested_last_updated_state', 'tested_source_state',
        'notesForState', 'cumulativeConfirmedNumberForState', 'cumulativeDeceasedNumberForState',
        'cumulativeRecoveredNumberForState', 'cumulativeTestedNumberForState', 'cumulativeVaccinatedNumberForState'
    ]

    states_data = states_data.rename(
        columns={0: "District", 1: "cumulativeConfirmedNumberForDistrict", 2: "cumulativeRecoveredNumberForDistrict",
                 3: "cumulativeDeceasedNumberForDistrict"})

    states_data["Date"] = date#str(datetime.datetime.now().date())

    states_data["State/UTCode"] = state

    states_data['cumulativeConfirmedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "t_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeDeceasedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "death_case"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))

    states_data['cumulativeRecoveredNumberForState'] = int(
        sum_soup.findAll("div", {"class": "discharge"})[0].findAll("span", {"class": "icount"})[0].getText().replace(
            ",", ""))
    
    
    states_data["last_updated"] = str(datetime.datetime.now())

    states_data['cumulativeTestedNumberForState'] = int(
        sum_soup.findAll("div", {"class": "testing_result"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    
    states_data['tested_last_updated_state'] = sum_soup.findAll(
        "div", {"class": "test_title"})[0].getText().strip().split("up to ")[-1]
    
    states_data['deltaTestedForState'] = int(
        sum_soup.findAll("div", {"class": "testing_sample"})[0].findAll("strong")[0].getText().replace(",", ""))
    
    for vcount in sum_soup.findAll("div", {"class": "total-vcount"}):
        if 'yday' not in vcount.attrs['class']:
            states_data['cumulativeVaccinatedNumberForState'] = int(vcount.findAll("strong")[0].getText().replace(",", ""))

    # states_data = states_data.reindex(columns=final_df_col)
    return states_data

In [13]:
today = (datetime.datetime.now() - timedelta.Timedelta(days=0)).date()
pDate = (datetime.datetime.now() - timedelta.Timedelta(days=1)).date()
print(str(today))
print(str(pDate))
TT_df = india("TT",str(today))

# ASHA .. For Running previous dates
# date_time_str = '2021/11/06'
# date_time_obj = datetime.datetime.strptime(date_time_str, '%Y/%m/%d')
# print(str(date_time_obj.date()))
# today = (date_time_obj - timedelta.Timedelta(days=0)).date()
# pDate = (date_time_obj - timedelta.Timedelta(days=1)).date()
# print(str(today))
# print(str(pDate))
# TT_df = india("TT",str(today))
# ASHA .. For Running previous dates



2021-11-20
2021-11-19


In [14]:
if not os.path.isdir(os.path.join("..","RAWCSV",str(pDate),"myGov")):
    os.mkdir(os.path.join("..","RAWCSV",str(pDate),"myGov"))
    print("Created")

Created


In [15]:
#mygov of prevDate
from ExtractStateMyGov import ExtractStateMyGov
source = pd.read_csv(r"../sources.csv")
for idx in source.index:
    if source["StateCode"][idx] != "TT":
        print(source["StateCode"][idx])
        if source["myGov"][idx] == "yes":
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today))
        else:
            temp_df = ExtractStateMyGov(source["StateCode"][idx],str(today), no_source = True)
        temp_df["Date"] = pDate
        temp_df.to_csv(os.path.join("..","RAWCSV",str(pDate),"myGov",source["StateCode"][idx]+"_raw.csv"))

AP
AR
AS


/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeConfirmedNumberForState"] = state_df['cumulativeConfirmedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeConfirmedNumberForDistrict"][ind["District"].str.contains(state_name)].values[0])
/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df["cumulativeDeceasedNumberForState"] = state_df['cumulativeDeceasedNumberForDistrict'][state_df['District'] == 'Unknown'] = int(ind["cumulativeDeceasedNumberForDis

BR
CT
GA
GJ
HR
HP
JH
KA
KL
MP
MH
MN
ML
MZ
NL


/home/swiadmin/Incovid19/incovid19/src/ExtractStateMyGov.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df['District'] == district] = None


OR
PB
RJ
SK
TN
TG
TR
UP
UT
WB
AN
CH
DN
DL
LA
LD
JK
PY


In [16]:
# TT_df = pd.read_csv("../RAWCSV/2021-10-31/TT_raw.csv")

In [17]:
TT_df = TT_df.dropna(1)

In [18]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(today)) as url:
    data = json.loads(url.read().decode())

In [19]:
TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [20]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [21]:
VAC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [22]:
VAC_df = VAC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [23]:
TT_df = TT_df.merge(VAC_df,how="left",on="District")

In [24]:
TT_df["tested_source_state"] = "https://www.icmr.gov.in/"

In [25]:
# TT_df.to_csv("TT_test.csv")

In [26]:
TT_df["tested_last_updated_state"] = str(datetime.datetime.strptime(TT_df["tested_last_updated_state"][0] , '%b %d, %Y'))

# Generate TT_final

In [27]:
# cs = pd.read_csv("case_time_series.csv")

In [28]:
# cs.columns

In [29]:
# cs.tail()

In [30]:
# import timedelta
# delta_date = str((datetime.datetime.now() - timedelta.Timedelta(days=1)).date())

In [31]:
delta_date = str(pDate)

In [32]:
delta_date

'2021-11-19'

In [33]:
# ind = cs[cs['Date_YMD']==delta_date].index[0]

# Integrate is CS only

In [34]:
# states_df = pd.read_csv("states.csv")

In [35]:
# cs = states_df

In [36]:
# ind = cs[(cs['Date']==delta_date) & (cs["State"] == "India")].index[0]

In [37]:
# ind

# Reading from Raw

In [38]:
cs = pd.read_csv("../RAWCSV/"+delta_date+"/TT_final.csv")

In [39]:
# cs.columns

In [40]:
# TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["Confirmed"][ind]
# TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["Deceased"][ind]
# TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["Recovered"][ind]
TT_df["deltaConfirmedForState"] = TT_df["cumulativeConfirmedNumberForState"] - cs["cumulativeConfirmedNumberForState"][0]
TT_df["deltaDeceasedForState"] = TT_df["cumulativeDeceasedNumberForState"] - cs["cumulativeDeceasedNumberForState"][0]
TT_df["deltaRecoveredForState"] = TT_df["cumulativeRecoveredNumberForState"] - cs["cumulativeRecoveredNumberForState"][0]
TT_df["deltaTestedForState"] = TT_df["cumulativeTestedNumberForState"] - cs["cumulativeTestedNumberForState"][0]

In [41]:
import urllib.request, json 
with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+delta_date) as url:
    data = json.loads(url.read().decode())

In [42]:
# TT_df["cumulativeVaccinated1NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_1"]
# TT_df["cumulativeVaccinated2NumberForState"] = data["topBlock"]["vaccination"]["tot_dose_2"]
# TT_df["cumulativeVaccinatedNumberForState"] = data["topBlock"]["vaccination"]["total_doses"]

In [43]:
TT_df["deltaVaccinated2ForState"] = TT_df["cumulativeVaccinated2NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_2"]
TT_df["deltaVaccinated1ForState"] = TT_df["cumulativeVaccinated1NumberForState"] - data["topBlock"]["vaccination"]["tot_dose_1"]
TT_df["deltaVaccinatedForState"] = TT_df["cumulativeVaccinatedNumberForState"] - data["topBlock"]["vaccination"]["total_doses"]

In [44]:
# TT_df["deltaVaccinatedForState"]

In [45]:
states = []
vaccinated = []
vaccinated1 = []
vaccinated2 = []
for sta in data['getBeneficiariesGroupBy']:
    if (sta['title'].lower() == "dadra and nagar haveli") or (sta['title'].lower() == "daman and diu"):
        states.append('Dadra and Nagar Haveli and Daman and Diu')
    else:
        states.append(sta['title'])
    vaccinated.append(sta['total'])
    vaccinated1.append(sta['partial_vaccinated'])
    vaccinated2.append(sta['totally_vaccinated'])

In [46]:
prevdayVACC_df = pd.DataFrame(list(zip(states,vaccinated,vaccinated1,vaccinated2))).groupby(0, as_index=False).sum()

In [47]:
prevdayVACC_df = prevdayVACC_df.rename(columns={0:"District",1:"cumulativeVaccinatedNumberForDistrict",2:"cumulativeVaccinated1NumberForDistrict",3:"cumulativeVaccinated2NumberForDistrict",})

In [48]:
# TT_df.to_csv("in_test.csv")

In [49]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [50]:
# states_df = pd.read_csv("states.csv")

# No reading states

In [51]:
states_df = cs[["Date","District","cumulativeConfirmedNumberForDistrict","cumulativeRecoveredNumberForDistrict","cumulativeDeceasedNumberForDistrict"]]

In [52]:
states_df

,Date,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict
0,2021-11-19,Andaman and Nicobar Islands,7675,7541,129
1,2021-11-19,Andhra Pradesh,2070738,2053755,14423
2,2021-11-19,Arunachal Pradesh,55242,54908,280
3,2021-11-19,Assam,614863,605656,6065
4,2021-11-19,Bihar,726178,716476,9663
5,2021-11-19,Chandigarh,65394,64546,820
6,2021-11-19,Chhattisgarh,1006459,992596,13591
7,2021-11-19,Dadra and Nagar Haveli and Daman and Diu,10682,10678,4
8,2021-11-19,Delhi,1440575,1415118,25095
9,2021-11-19,Goa,178594,174965,3377


In [53]:
# deltaConfirmedForDistrict",
#                  "deltaDeceasedForDistrict",
#                  "deltaRecoveredForDistrict",
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        # print(TT_df["cumulative{}NumberForDistrict".format(val)][idx])
        # print(TT_df["District".format(val)][idx])
        # print(delta_date)
        # print(int(TT_df["cumulative{}NumberForDistrict".format(val)][idx]) - (states_df[(states_df["District"] == TT_df["District"][idx]) & (states_df["Date"] == delta_date)]["cumulative{}NumberForDistrict".format(val)])) - cumulative{}NumberForDistrict
        TT_df["delta{}ForDistrict".format(val)][idx] = int(TT_df["cumulative{}NumberForDistrict".format(val)][idx]) - int(states_df[(states_df["District"] == TT_df["District"][idx]) & (states_df["Date"] == delta_date)]["cumulative{}NumberForDistrict".format(val)].item())

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [54]:
lst = ["Vaccinated1","Vaccinated2","Vaccinated"]
for val in lst:
    TT_df["delta{}ForDistrict".format(val)] = None
    for idx in TT_df.index:
        TT_df["delta{}ForDistrict".format(val)][idx] = TT_df["cumulative{}NumberForDistrict".format(val)][idx] - prevdayVACC_df[prevdayVACC_df["District"] == TT_df["District"][idx]]["cumulative{}NumberForDistrict".format(val)].item()

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [55]:
# TT_df 7dma Cummulative

In [56]:
cs.tail()

,Unnamed: 0,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaConfirmedForState,7DmaRecoveredForState,7DmaDeceasedForState,7DmaVaccinated1ForState,7DmaVaccinated2ForState,7DmaVaccinatedForState,7DmaConfirmedForDistrict,7DmaRecoveredForDistrict,7DmaDeceasedForDistrict,7DmaVaccinatedForDistrict,7DmaVaccinated1ForDistrict,7DmaVaccinated2ForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation,StateName,districtPopulation
31,31,Telangana,674181,666509,3978,2021-11-19,TT,34489623,465082,33897921,2021-11-19 06:34:24.847044,629387540,2021-11-18 00:00:00,1138699,1153595556,762169283,391426273,35739528,24070740,11668788,https://www.icmr.gov.in/,11106,459,12789,687929,312644,1000573,13303,23687,36990,144,161,1,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Telangana,37220000
32,32,Tripura,84698,83793,818,2021-11-19,TT,34489623,465082,33897921,2021-11-19 06:34:24.847044,629387540,2021-11-18 00:00:00,1138699,1153595556,762169283,391426273,4434560,2528901,1905659,https://www.icmr.gov.in/,11106,459,12789,687929,312644,1000573,224,2810,3034,6,15,1,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Tripura,3992000
33,33,Uttar Pradesh,1710306,1687296,22909,2021-11-19,TT,34489623,465082,33897921,2021-11-19 06:34:24.847044,629387540,2021-11-18 00:00:00,1138699,1153595556,762169283,391426273,145606199,104187077,41419122,https://www.icmr.gov.in/,11106,459,12789,687929,312644,1000573,65725,85644,151369,12,6,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Uttar Pradesh,224979000
34,34,Uttarakhand,344074,336491,7404,2021-11-19,TT,34489623,465082,33897921,2021-11-19 06:34:24.847044,629387540,2021-11-18 00:00:00,1138699,1153595556,762169283,391426273,12073237,7532646,4540591,https://www.icmr.gov.in/,11106,459,12789,687929,312644,1000573,681,8125,8806,12,6,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,NaN,NaN,1332900000,Uttarakhand,11141000
35,35,West Bengal,1607516,1580089,19355,2021-11-19,TT,34489623,465082,33897921,2021-11-19 06:34:24.847044,629387540,2021-11-18 00:00:00,1138699,1153595556,762169283,391426273,86352229,60924063,25428166,https://www.icmr.gov.in/,11106,459,12789,687929,312644,1000573,28511,35102,63613,860,825,14,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,76759973,55332210,21427762,NaN,NaN,1332900000,West Bengal,96906000


In [57]:
cs = pd.read_csv("case_time_series.csv")

In [58]:
states_df = pd.read_csv("states.csv")

# Remenber to read CS and states

In [59]:
from datetime import datetime, timedelta
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = 0
    for i in range(7):
        if i == 0:
            value = TT_df["cumulative{}NumberForState".format(val)]
        else:
            prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
            ind = cs[cs['Date_YMD']==str(prev_date.date())].index[0]
            value =+ cs["Total {}".format(val)][ind]
    TT_df["7Dma{}ForState".format(val)] = value/7

In [60]:
import urllib.request, json 
va1 = 0
va2 = 0
va = 0
for i in range(7):
    prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
    with urllib.request.urlopen("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?date="+str(prev_date.date())) as url:
        data = json.loads(url.read().decode())
        va1 += data["topBlock"]["vaccination"]["tot_dose_1"]
        va2 += data["topBlock"]["vaccination"]["tot_dose_2"]
        va += data["topBlock"]["vaccination"]["total_doses"]


In [61]:
TT_df["7DmaVaccinated1ForState"] = va1
TT_df["7DmaVaccinated2ForState"] = va2
TT_df["7DmaVaccinatedForState"] = va

In [62]:
# 7dma District

In [63]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = 0
    for idx in TT_df.index:
        for i in range(7):
            if i == 0:
                value = int(TT_df["cumulative{}NumberForDistrict".format(val)][0])
            else:
                prev_date=datetime.strptime("2021-10-31","%Y-%m-%d")+timedelta(days=(-1*i))
                value += int(states_df[(states_df["State"] == TT_df["District"][idx]) & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())
    TT_df["7Dma{}ForDistrict".format(val)] = value/7          

In [64]:
cowin = pd.read_csv("cowin_vaccine_data_statewise.csv")

In [65]:
va1 = 0
va2 = 0
va = 0
TT_df["7DmaVaccinatedForDistrict"] = 0
TT_df["7DmaVaccinated1ForDistrict"] = 0
TT_df["7DmaVaccinated2ForDistrict"] = 0
for idx in TT_df.index:
    for i in range(7):
        if i == 0:
            va = int(TT_df["cumulativeVaccinatedNumberForDistrict"][idx])
            va1 = int(TT_df["cumulativeVaccinated1NumberForDistrict"][idx])
            va2 = int(TT_df["cumulativeVaccinated2NumberForDistrict"][idx])
        else:
            prev_date=datetime.strptime("2021/10/31","%Y/%m/%d")+timedelta(days=(-1*i))
            va += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Total Doses Administered"].item())
            va1 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["First Dose Administered"].item())
            va2 += int(cowin[(cowin["Updated On"] ==  prev_date.strftime('%d/%m/%Y')) & (cowin["State"] == TT_df["District"][idx])]["Second Dose Administered"].item())
TT_df["7DmaVaccinatedForDistrict"][idx] = va/7
TT_df["7DmaVaccinated1ForDistrict"][idx] = va1/7
TT_df["7DmaVaccinated2ForDistrict"][idx] = va2/7

/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [66]:
TT_df["7DmaTestedForState"] = None
TT_df["delta21_14confirmedForState"] = None

In [67]:
population = pd.read_csv("../CSV/StatePopulation.csv")

In [68]:
sources = pd.read_csv("../sources.csv")

In [69]:
population = population.merge(sources,how="left",left_on = "State", right_on = "StateCode")

In [70]:
population

,State,StatePop,StateName,StateCode,StateDataSourceType,myGov,StateDataURL,Twitter Handle,Twitter Search Term
0,AN,397000,Andaman and Nicobar Islands,AN,html,yes,https://www.mygov.in/covid-19,NaN,NaN
1,AP,52221000,Andhra Pradesh,AP,html,NaN,https://hmfw.ap.gov.in/covid_dashboard.aspx,NaN,NaN
2,AR,1504000,Arunachal Pradesh,AR,Image(Twitter),NaN,https://twitter.com/DirHealth_ArPr,DirHealth_ArPr,#ArunachalCoronaUpdate
3,AS,34293000,Assam,AS,html,yes,https://www.mygov.in/covid-19,NaN,NaN
4,BR,119520000,Bihar,BR,Image(Twitter),NaN,https://twitter.com/BiharHealthDept,BiharHealthDept,#COVIDー19 Updates Bihar
5,CH,1179000,Chandigarh,CH,html,yes,https://www.mygov.in/covid-19,NaN,NaN
6,CT,28724000,Chhattisgarh,CT,Image(Twitter),NaN,https://twitter.com/HealthCgGov,HealthCgGov,#ChhattisgarhFightsCorona
7,DL,19814000,Delhi,DL,html,yes,https://www.mygov.in/covid-19,NaN,NaN
8,DN,959000,Dadra and Nagar Haveli and Daman and Diu,DN,html,yes,https://www.mygov.in/covid-19,NaN,NaN
9,GA,1540000,Goa,GA,html,yes,https://www.mygov.in/covid-19,NaN,NaN


In [71]:
population[["StateName","StatePop"]]

,StateName,StatePop
0,Andaman and Nicobar Islands,397000
1,Andhra Pradesh,52221000
2,Arunachal Pradesh,1504000
3,Assam,34293000
4,Bihar,119520000
5,Chandigarh,1179000
6,Chhattisgarh,28724000
7,Delhi,19814000
8,Dadra and Nagar Haveli and Daman and Diu,959000
9,Goa,1540000


In [72]:
TT_df["statePopulation"] = population[population["StateName"] == "India"]["StatePop"].item()

In [73]:
TT_df

,District,cumulativeConfirmedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeDeceasedNumberForDistrict,Date,State/UTCode,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,last_updated,cumulativeTestedNumberForState,tested_last_updated_state,deltaTestedForState,cumulativeVaccinatedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForDistrict,cumulativeVaccinated1NumberForDistrict,cumulativeVaccinated2NumberForDistrict,tested_source_state,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaVaccinated2ForState,deltaVaccinated1ForState,deltaVaccinatedForState,deltaVaccinated1ForDistrict,deltaVaccinated2ForDistrict,deltaVaccinatedForDistrict,deltaConfirmedForDistrict,deltaRecoveredForDistrict,deltaDeceasedForDistrict,7DmaConfirmedForState,7DmaRecoveredForState,7DmaDeceasedForState,7DmaVaccinated1ForState,7DmaVaccinated2ForState,7DmaVaccinatedForState,7DmaConfirmedForDistrict,7DmaRecoveredForDistrict,7DmaDeceasedForDistrict,7DmaVaccinatedForDistrict,7DmaVaccinated1ForDistrict,7DmaVaccinated2ForDistrict,7DmaTestedForState,delta21_14confirmedForState,statePopulation
0,Andaman and Nicobar Islands,7676,7541,129,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,530327,295693,234634,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,0,54,54,1,0,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,None,None,1332900000
1,Andhra Pradesh,2070906,2054056,14425,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,56936990,33900152,23036838,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,522,2499,3021,168,301,2,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,None,None,1332900000
2,Arunachal Pradesh,55243,54915,280,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,1370458,784733,585725,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,47,107,154,1,7,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,None,None,1332900000
3,Assam,615053,605933,6066,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,31309654,20732286,10577368,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,1808,3135,4943,190,277,1,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,None,None,1332900000
4,Bihar,726182,716480,9663,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,74275333,52168512,22106821,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,10069,11665,21734,4,4,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,None,None,1332900000
5,Chandigarh,65398,64551,820,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,1530893,934217,596676,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,12,85,97,4,5,0,4.885883e+06,4.796504e+06,64930.0,5094504913,2239844681,7334349594,1.363443e+06,1.340164e+06,16386.142857,0,0,0,None,None,1332900000
6,Chhattisgarh,1006494,992613,13591,2021-11-20,TT,34499925,465349,33909708,2021-11-20 05:13:59.602055,630575279,2021-11-19 00:00:00,1187739,1158433611,763834772,394598839,24156242,15617627,8538615,https://www.icmr.gov.in/,10302,267,11787,108266,80872,189138,4403,2438,6841,35,17,0,4.885883e+06,4.796504e+06,64930.0,5094504913,223984

In [74]:
TT_df = TT_df.merge(population[["StateName","StatePop"]],how="left",left_on = "District", right_on = "StateName")

In [75]:
TT_df = TT_df.rename(columns={"StatePop":"districtPopulation"})

In [76]:
today

datetime.date(2021, 11, 20)

In [77]:
TT_df["Date"] = str(today)

In [78]:
TT_df.to_csv('../RAWCSV/'+str(today)+'/TT_final.csv')

In [79]:
TT_df["Date"] = str(pDate)

In [80]:
TT_df.to_csv('../RAWCSV/'+str(pDate)+'/TT_final.csv')

# Stop Here

In [348]:
# 7dma delta

In [297]:
states_df[states_df["State"] == "India"]

,Date,State,Confirmed,Recovered,Deceased,Other,Tested
1,2020-01-30,India,1,0,0,0,NaN
3,2020-02-02,India,2,0,0,0,NaN
5,2020-02-03,India,3,0,0,0,NaN
7,2020-02-14,India,3,3,0,0,NaN
11,2020-03-02,India,5,3,0,0,NaN
...,...,...,...,...,...,...,...
21523,2021-10-27,India,34231243,33606777,456418,13182,604498405.0
21560,2021-10-28,India,34245550,33619966,457223,13187,605885769.0
21597,2021-10-29,India,34259765,33633515,457774,13195,607062619.0
21634,2021-10-30,India,34272705,33648187,458219,13196,608319915.0


In [298]:
lst = ["Confirmed","Recovered","Deceased"]
for val in lst:
    value = []
    for i in range(7):
        if i == 0:
            value.append(TT_df["cumulative{}NumberForState".format(val)][0])
        else:
            prev_date=datetime.strptime("2021-11-01","%Y-%m-%d")+timedelta(days=(-1*i))
            value.append(states_df[(states_df["State"] == "India") & (states_df["Date"] == str(prev_date.date()))]['{}'.format(val)].item())

In [299]:
value

[458437, 458470, 458219, 457774, 457223, 456418, 455684]